In [5]:
import os
import pandas as pd
import numpy as np
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import copy
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import joblib
import optuna
from tqdm import tqdm
import time
import codecs
import pickle

# Configuration

In [6]:
rawdata_dir = 'rawdata'
data_dir = 'data'

# lags = [1,2,3,6,12,24,36]
lags = [28, 35, 42, 49, 56]

# NUM_FOLD_DAYS = 28
NUM_FOLD_DAYS = 56
VALID_FIRST_DAY = 1914
VALID_LAST_DAY = 1941
NUM_USE_DAY = 365
# TRAIN_FIRST_DAY = max(lags)
TRAIN_FIRST_DAY = VALID_FIRST_DAY - NUM_USE_DAY + 1

OPTUNA_TRIAL = 70

# Read data

In [7]:
sales = pd.read_csv(os.path.join(rawdata_dir, 'sales_train_evaluation.csv'))
sales.name = 'sales'
calendar = pd.read_csv(os.path.join(rawdata_dir, 'calendar.csv'))
calendar.name = 'calendar'
prices = pd.read_csv(os.path.join(rawdata_dir, 'sell_prices.csv'))
prices.name = 'prices'

Since, the validation data is now available for the days 1914-1941, Adding zero sales for days: d_1942 - d_1969(Test)

In [8]:
#Add zero sales for the remaining days 1942-1969
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)

# Downcast

In [9]:
#Downcast in order to save memory
def downcast(df):
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

# Melt data

In [10]:
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()

## Combine data
Combine price data from prices dataframe and days data from calendar dataset.

In [11]:
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left')

# Feature Engineering

## Label Encoding

In [12]:
#Store the categories along with their codes
d_id = dict(zip(df.id.cat.codes, df.id))
d_item_id = dict(zip(df.item_id.cat.codes, df.item_id))
d_dept_id = dict(zip(df.dept_id.cat.codes, df.dept_id))
d_cat_id = dict(zip(df.cat_id.cat.codes, df.cat_id))
d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))
d_state_id = dict(zip(df.state_id.cat.codes, df.state_id))

In [13]:
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes
        
df.drop('date',axis=1,inplace=True)

## Introduce lags

In [14]:
#Introduce lags
def make_lags(df, lags):
    for lag in lags:
        df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag).astype(np.float16)
        
    return df

In [15]:
df = make_lags(df, lags)

## make new features

In [16]:
def make_new_feature(df, d_last = df['d'].max()):
    # copy sold
    sold_tmp = copy.copy(df['sold'])
    
    # embed NaN
    df['sold'][df['d'] > d_last] = np.nan
    
    # make feature using sold
    df['item_sold_avg'] = df.groupby('item_id')['sold'].transform('mean').astype(np.float16)
    df['state_sold_avg'] = df.groupby('state_id')['sold'].transform('mean').astype(np.float16)
    df['store_sold_avg'] = df.groupby('store_id')['sold'].transform('mean').astype(np.float16)
    df['cat_sold_avg'] = df.groupby('cat_id')['sold'].transform('mean').astype(np.float16)
    df['dept_sold_avg'] = df.groupby('dept_id')['sold'].transform('mean').astype(np.float16)
    df['cat_dept_sold_avg'] = df.groupby(['cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
    df['store_item_sold_avg'] = df.groupby(['store_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['cat_item_sold_avg'] = df.groupby(['cat_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['dept_item_sold_avg'] = df.groupby(['dept_id','item_id'])['sold'].transform('mean').astype(np.float16)
    df['state_store_sold_avg'] = df.groupby(['state_id','store_id'])['sold'].transform('mean').astype(np.float16)
    df['state_store_cat_sold_avg'] = df.groupby(['state_id','store_id','cat_id'])['sold'].transform('mean').astype(np.float16)
    df['store_cat_dept_sold_avg'] = df.groupby(['store_id','cat_id','dept_id'])['sold'].transform('mean').astype(np.float16)
    
    df['rolling_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.rolling(window=7).mean()).astype(np.float16)
    df['expanding_sold_mean'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform(lambda x: x.expanding(2).mean()).astype(np.float16)
    
    df['daily_avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','d'])['sold'].transform('mean').astype(np.float16)
    df['avg_sold'] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold'].transform('mean').astype(np.float16)
    df['selling_trend'] = (df['daily_avg_sold'] - df['avg_sold']).astype(np.float16)
    df.drop(['daily_avg_sold','avg_sold'],axis=1,inplace=True)
    
    # restore sold
    df['sold'] = sold_tmp
    
    return df

In [13]:
df_all = copy.copy(df)
df_all = make_new_feature(df_all)
df_all = df_all[df_all['d'] > max(lags)]
df_all.info()

# df_all.to_pickle(os.path.join(data_dir, 'data_all.pkl'))   # save df_all (use this after CV)
df_all.to_pickle(os.path.join(data_dir, 'data_all_lagOlder28Modified.pkl'))   # save df_all (use this after CV)
del df_all
gc.collect()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58357860 entries, 1676950 to 60034809
Data columns (total 41 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   id                        int16  
 1   item_id                   int16  
 2   dept_id                   int8   
 3   cat_id                    int8   
 4   store_id                  int8   
 5   state_id                  int8   
 6   d                         int16  
 7   sold                      int16  
 8   wm_yr_wk                  int16  
 9   weekday                   int8   
 10  wday                      int8   
 11  month                     int8   
 12  year                      int16  
 13  event_name_1              int8   
 14  event_type_1              int8   
 15  event_name_2              int8   
 16  event_type_2              int8   
 17  snap_CA                   int8   
 18  snap_TX                   int8   
 19  snap_WI                   int8   
 20  sell_price      

0

# Cross Validation

## Cross Validation for each store (function)

In [14]:
def xgboost_cv_store(df, store):
    def objective(trial):
        # param space
        param = {
            'objective': 'reg:squarederror',
            'n_estimators': 1000,
            'learning_rate': 0.1,
            'subsample': trial.suggest_discrete_uniform('subsample', 0.6, 0.95, 0.05),
            'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.6, 0.95, 0.05),
            'max_depth':  trial.suggest_int('max_depth', 3, 9),
            'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
            'min_child_weight': trial.suggest_loguniform('min_child_weight', 0.1, 10.0),
            'tree_method': 'gpu_hist'
        }
        
        ### START: CV Loop ###
        isBreakCV = False
        fold_iter = 0
        rmse_mean = 0.
        while not(isBreakCV):
            start = time.time()   # start timer
            
            d_valid_last = VALID_LAST_DAY - NUM_FOLD_DAYS * fold_iter
            d_train_last = VALID_LAST_DAY - NUM_FOLD_DAYS * (fold_iter + 1)
            if d_train_last < TRAIN_FIRST_DAY + NUM_FOLD_DAYS * 2:
                isBreakCV = True
            
            # make_new_feature
#             df_fold_file = os.path.join(data_dir, 'data_foldday'+str(NUM_FOLD_DAYS)+'_fold'+str(fold_iter + 1)+'.pkl')
            df_fold_file = os.path.join(data_dir, 'data_foldday'+str(NUM_FOLD_DAYS)+'_fold'+str(fold_iter + 1)+'_lagOlder28Modified.pkl')
            
            if os.path.isfile(df_fold_file):   # make_new_feature 適用後のファイルがあればそれを使う
                df_fold = pd.read_pickle(df_fold_file)
            else:
                df_fold = df[df['d'] <= d_valid_last]
                df_fold = make_new_feature(df_fold, d_train_last)   # train の情報だけを使って特徴量を作成
                df_fold.to_pickle(df_fold_file)   # pickleとして保存しておく

            # Extract store
            df_fold = df_fold[df_fold['store_id'] == store]
                        
            # Split data
            X_train = df_fold[(df_fold['d'] >= TRAIN_FIRST_DAY) & (df_fold['d'] <= d_train_last)].drop('sold', axis=1)
            y_train = df_fold[(df_fold['d'] >= TRAIN_FIRST_DAY) & (df_fold['d'] <= d_train_last)]['sold']
            X_valid = df_fold[(df_fold['d'] > d_train_last) & (df_fold['d'] <= d_valid_last)].drop('sold', axis=1)
            y_valid = df_fold[(df_fold['d'] > d_train_last) & (df_fold['d'] <= d_valid_last)]['sold']
            
            # Train XGBoost model
            model = xgb.XGBRegressor(**param)
#             model.fit(X_train, y_train, eval_set=[(X_train,y_train), (X_valid, y_valid)],
#                  eval_metric='rmse', verbose=10, early_stopping_rounds=100)
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
                  eval_metric='rmse', verbose=10, early_stopping_rounds=50)
            y_pred = model.predict(X_valid)
            rmse_mean += np.sqrt(mean_squared_error(y_valid.values, y_pred))
            
            fold_iter += 1
            elapsed_time = time.time() - start   # stop timer
            print("[{0}] Fold{1}: {2} [sec]".format(d_store_id[store], fold_iter, elapsed_time))
        ### END: CV Loop ###
        
        del df_fold, model, X_train, y_train, X_valid, y_valid, y_pred
        gc.collect()
        
        rmse_mean /= fold_iter
        return rmse_mean
    return objective


## Execute CV

In [15]:
stores = sales.store_id.cat.codes.unique().tolist()

# find best params by doing CV (optuna)
for store in tqdm(stores[6:]):
    print('##################### CV START: {0} #####################'.format(d_store_id[store]))
    study = optuna.create_study()
    study.optimize(xgboost_cv_store(df, store), n_trials=OPTUNA_TRIAL)
    
    # save optuna log
    df_trial = study.trials_dataframe()
#     df_trial.to_pickle(os.path.join('report', 'OptunaTrials_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'))
    df_trial.to_pickle(os.path.join('report', 'OptunaTrials_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'_lagOlder28Modified.pkl'))
    
    del df_trial
    gc.collect()
    
    # savae best params found by CV
#     bestparam_filename = 'BestParamsSelectedByCV_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    bestparam_filename = 'BestParamsSelectedByCV_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'_lagOlder28Modified.pkl'
    with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'wb') as f:
        pickle.dump(study.best_params, f)


  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

##################### CV START: TX_3 #####################
[0]	validation_0-rmse:3.33812
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:7.63299
[20]	validation_0-rmse:13.47191
[30]	validation_0-rmse:14.70081
[40]	validation_0-rmse:15.94455
[50]	validation_0-rmse:16.73743
Stopping. Best iteration:
[5]	validation_0-rmse:2.73317

[TX_3] Fold1: 247.18861055374146 [sec]
[0]	validation_0-rmse:3.51775
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:7.53044
[20]	validation_0-rmse:13.05251
[30]	validation_0-rmse:15.56291
[40]	validation_0-rmse:16.08235
[50]	validation_0-rmse:17.47402
Stopping. Best iteration:
[4]	validation_0-rmse:3.04985

[TX_3] Fold2: 256.78028678894043 [sec]
[0]	validation_0-rmse:3.18801
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:10.40211
[20]	validation_0-rmse:12.85687
[30]	validation_0-rmse:15.35599
[40]	validation_0-rmse:17.05017
[50]	validation_0-

[I 2020-06-29 01:15:33,368] Finished trial#0 with value: 2.831009531021118 with parameters: {'subsample': 0.8, 'colsample_bytree': 0.6, 'max_depth': 5, 'gamma': 4.278805663373544e-07, 'min_child_weight': 6.883970285272298}. Best is trial#0 with value: 2.831009531021118.


[0]	validation_0-rmse:3.33118
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:11.11620
[20]	validation_0-rmse:16.85883
[30]	validation_0-rmse:24.49419
[40]	validation_0-rmse:26.16360
[50]	validation_0-rmse:26.15979
Stopping. Best iteration:
[1]	validation_0-rmse:3.14121

[TX_3] Fold1: 27.897847175598145 [sec]
[0]	validation_0-rmse:3.51168
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:5.43688
[20]	validation_0-rmse:9.10994
[30]	validation_0-rmse:14.87758
[40]	validation_0-rmse:14.96588
[50]	validation_0-rmse:16.92859
Stopping. Best iteration:
[6]	validation_0-rmse:3.11009

[TX_3] Fold2: 11.338099479675293 [sec]
[0]	validation_0-rmse:3.20232
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:2.81559
[20]	validation_0-rmse:2.81561
[30]	validation_0-rmse:4.60489
[40]	validation_0-rmse:8.44093
[50]	validation_0-rmse:10.35607
[60]	validation_0-rmse:13.55806
[70]	validation_0

[I 2020-06-29 01:16:38,025] Finished trial#1 with value: 3.077878952026367 with parameters: {'subsample': 0.6, 'colsample_bytree': 0.7, 'max_depth': 5, 'gamma': 4.848352364514525e-07, 'min_child_weight': 1.155127332198429}. Best is trial#0 with value: 2.831009531021118.


...

[I 2020-06-29 05:37:35,168] Finished trial#67 with value: 2.947301483154297 with parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'max_depth': 8, 'gamma': 6.129247036539405e-08, 'min_child_weight': 8.47490125724481}. Best is trial#27 with value: 2.9296722412109375.


[0]	validation_0-rmse:3.65288
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:7.70151
[20]	validation_0-rmse:8.43743
[30]	validation_0-rmse:9.80495
[40]	validation_0-rmse:9.83298
[50]	validation_0-rmse:10.86631
Stopping. Best iteration:
[5]	validation_0-rmse:3.10291

[WI_3] Fold1: 11.429295063018799 [sec]
[0]	validation_0-rmse:3.84460
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:12.73364
[20]	validation_0-rmse:20.97522
[30]	validation_0-rmse:24.65537
[40]	validation_0-rmse:24.37338
[50]	validation_0-rmse:25.55305
Stopping. Best iteration:
[5]	validation_0-rmse:3.23919

[WI_3] Fold2: 15.831092596054077 [sec]
[0]	validation_0-rmse:3.38408
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:7.22817
[20]	validation_0-rmse:15.81515
[30]	validation_0-rmse:20.40537
[40]	validation_0-rmse:20.23444
[50]	validation_0-rmse:20.06680
Stopping. Best iteration:
[6]	validation_0-rmse:

[I 2020-06-29 05:38:36,518] Finished trial#68 with value: 2.947301483154297 with parameters: {'subsample': 0.9, 'colsample_bytree': 0.6, 'max_depth': 8, 'gamma': 7.784120946557929e-08, 'min_child_weight': 8.187452231176394}. Best is trial#27 with value: 2.9296722412109375.


[0]	validation_0-rmse:3.85776
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:17.90732
[20]	validation_0-rmse:20.92571
[30]	validation_0-rmse:20.98891
[40]	validation_0-rmse:19.82806
[50]	validation_0-rmse:20.36738
Stopping. Best iteration:
[0]	validation_0-rmse:3.85776

[WI_3] Fold1: 10.258885145187378 [sec]
[0]	validation_0-rmse:4.00040
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:9.75326
[20]	validation_0-rmse:19.09362
[30]	validation_0-rmse:18.51905
[40]	validation_0-rmse:18.22486
[50]	validation_0-rmse:18.05041
Stopping. Best iteration:
[1]	validation_0-rmse:3.98893

[WI_3] Fold2: 13.465459108352661 [sec]
[0]	validation_0-rmse:3.52765
Will train until validation_0-rmse hasn't improved in 50 rounds.
[10]	validation_0-rmse:10.28571
[20]	validation_0-rmse:12.19851
[30]	validation_0-rmse:14.56331
[40]	validation_0-rmse:15.70056
[50]	validation_0-rmse:17.97788
Stopping. Best iteration:
[0]	validation_0-r

[I 2020-06-29 05:39:31,545] Finished trial#69 with value: 3.67585711479187 with parameters: {'subsample': 0.9, 'colsample_bytree': 0.9, 'max_depth': 7, 'gamma': 2.826210988596218e-08, 'min_child_weight': 9.1569028860059}. Best is trial#27 with value: 2.9296722412109375.
100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [4:44:01<00:00, 4260.29s/it]


In [23]:
del df
gc.collect()

176

## Train & Predict by best model (function)

In [24]:
def xgboost_by_bestparam_for_valid(df, store, best_params, valid_preds):
    print('##################### Traing4ValidByBestParam START: {0} #####################'.format(d_store_id[store]))
    print('Best Parameters of store {0} model: {1}'.format(d_store_id[store], best_params))
    
    # prepare data set
    df_store = df[df['store_id'] == store]
    X_train = df_store[df_store['d'] < VALID_FIRST_DAY].drop('sold', axis=1)
    y_train = df_store[df_store['d'] < VALID_FIRST_DAY]['sold']
    X_valid = df_store[(df_store['d'] >= VALID_FIRST_DAY) & (df_store['d'] <= VALID_LAST_DAY)].drop('sold', axis=1)
    
    # train XGBoost by using best_params
    model = xgb.XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_metric='rmse')
    
    # save model for validation
#     modelfile = 'BestModel4valid_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    modelfile = 'BestModel4valid_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'_lagOlder28Modified.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    
    # predict for valid
    valid_preds[X_valid.index] = model.predict(X_valid)
    del df_store, X_train, y_train, X_valid, model
    gc.collect()
    

In [25]:
def xgboost_by_bestparam_for_test(df, store, best_params, eval_preds):
    print('##################### Traing4TestByBestParam START: {0} #####################'.format(d_store_id[store]))
    
    # prepare data set
    df_store = df[df['store_id'] == store]
    X_train = df_store[df_store['d'] <= VALID_LAST_DAY].drop('sold', axis=1)
    y_train = df_store[df_store['d'] <= VALID_LAST_DAY]['sold']
    X_test = df_store[df_store['d'] > VALID_LAST_DAY].drop('sold', axis=1)
    
    # train XGBoost by using best_params
    model = xgb.XGBRegressor(**best_params)
    model.fit(X_train, y_train, eval_metric='rmse')
        
    # predict for test
    eval_preds[X_test.index] = model.predict(X_test)
    
    # save model for test
#     modelfile = 'BestModel4test_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    modelfile = 'BestModel4test_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'_lagOlder28Modified.pkl'
    with open(os.path.join('result', 'best_model', modelfile), 'wb') as f:
        pickle.dump(model, f)
    
    del df_store, X_train, y_train, X_test, model
    gc.collect()
    

## Train & Predict by best model

In [26]:
# train & predict (best model)
# df_all = pd.read_pickle(os.path.join(data_dir, 'data_all.pkl'))
df_all = pd.read_pickle(os.path.join(data_dir, 'data_all_lagOlder28Modified.pkl'))

valid = df_all[(df_all['d'] >= VALID_FIRST_DAY) & (df_all['d'] <= VALID_LAST_DAY)][['id','d','sold']]
test = df_all[df_all['d'] > VALID_LAST_DAY][['id','d','sold']]
valid_preds = valid['sold']
eval_preds = test['sold']

for store in tqdm(stores):
    # read best param selected by CV
#     bestparam_filename = 'BestParamsSelectedByCV_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
    bestparam_filename = 'BestParamsSelectedByCV_'+d_store_id[store]+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'_lagOlder28Modified.pkl'
    with open(os.path.join('result', 'best_param_cv', bestparam_filename), 'rb') as f:
        best_params = pickle.load(f)

    # add other params
    best_params['objective'] = 'reg:squarederror'
    best_params['n_estimators'] = 1000
    best_params['learning_rate'] = 0.1
    best_params['tree_method'] = 'gpu_hist'
    
    # best model for valid
    xgboost_by_bestparam_for_valid(df_all, store, best_params, valid_preds)
    
    # best model for test
    xgboost_by_bestparam_for_test(df_all, store, best_params, eval_preds)


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

##################### Traing4ValidByBestParam START: CA_1 #####################
Best Parameters of store CA_1 model: {'subsample': 0.7, 'colsample_bytree': 0.6, 'max_depth': 7, 'gamma': 6.28612397986017e-08, 'min_child_weight': 0.2127772568587813, 'objective': 'reg:squarederror', 'n_estimators': 1000, 'learning_rate': 0.1, 'tree_method': 'gpu_hist'}
##################### Traing4TestByBestParam START: CA_1 #####################


 10%|████████▏                                                                         | 1/10 [02:35<23:20, 155.65s/it]

##################### Traing4ValidByBestParam START: CA_2 #####################
Best Parameters of store CA_2 model: {'subsample': 0.85, 'colsample_bytree': 0.6, 'max_depth': 9, 'gamma': 2.0094917326731126e-08, 'min_child_weight': 5.89417662259203, 'objective': 'reg:squarederror', 'n_estimators': 1000, 'learning_rate': 0.1, 'tree_method': 'gpu_hist'}
##################### Traing4TestByBestParam START: CA_2 #####################


 20%|████████████████▍                                                                 | 2/10 [06:17<23:23, 175.42s/it]

...

 80%|█████████████████████████████████████████████████████████████████▌                | 8/10 [24:49<06:36, 198.04s/it]

##################### Traing4ValidByBestParam START: WI_2 #####################
Best Parameters of store WI_2 model: {'subsample': 0.7, 'colsample_bytree': 0.65, 'max_depth': 8, 'gamma': 2.483792422624206e-07, 'min_child_weight': 4.6253137924147945, 'objective': 'reg:squarederror', 'n_estimators': 1000, 'learning_rate': 0.1, 'tree_method': 'gpu_hist'}
##################### Traing4TestByBestParam START: WI_2 #####################


 90%|█████████████████████████████████████████████████████████████████████████▊        | 9/10 [27:43<03:10, 190.92s/it]

##################### Traing4ValidByBestParam START: WI_3 #####################
Best Parameters of store WI_3 model: {'subsample': 0.65, 'colsample_bytree': 0.6, 'max_depth': 8, 'gamma': 0.006160450194643451, 'min_child_weight': 3.363429145836051, 'objective': 'reg:squarederror', 'n_estimators': 1000, 'learning_rate': 0.1, 'tree_method': 'gpu_hist'}
##################### Traing4TestByBestParam START: WI_3 #####################


100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [30:34<00:00, 183.50s/it]


In [27]:
# save valid_preds
# valid_preds_filename = 'ValidPreds_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
valid_preds_filename = 'ValidPreds_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'_lagOlder28Modified.pkl'
with open(os.path.join('result', 'preds', valid_preds_filename), 'wb') as f:
    pickle.dump(valid_preds, f)
    
# save test_preds
# eval_preds_filename = 'EvalPreds_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.pkl'
eval_preds_filename = 'EvalPreds_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'_lagOlder28Modified.pkl'
with open(os.path.join('result', 'preds', eval_preds_filename), 'wb') as f:
    pickle.dump(eval_preds, f)

In [28]:
del df_all
gc.collect()

44

# Make submission

In [30]:
#Set actual equal to false if you want to top in the public leaderboard :P
actual = True
if actual == False:
    #Get the validation results(We already have them as less than one month left for competition to end)
    validation = sales[['id']+['d_' + str(i) for i in range(1914,1942)]]
    validation['id']=pd.read_csv(os.path.join(rawdata_dir, 'sales_train_validation.csv')).id
    validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
else:
    #Get the actual validation results
    valid['sold'] = valid_preds
    validation = valid[['id','d','sold']]
    validation = pd.pivot(validation, index='id', columns='d', values='sold').reset_index()
    validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
    validation.id = validation.id.map(d_id).str.replace('evaluation','validation')

#Get the evaluation results
test['sold'] = eval_preds
evaluation = test[['id','d','sold']]
evaluation = pd.pivot(evaluation, index='id', columns='d', values='sold').reset_index()
evaluation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
#Remap the category id to their respective categories
evaluation.id = evaluation.id.map(d_id)

#Prepare the submission
submit = pd.concat([validation,evaluation]).reset_index(drop=True)

isNegative2Zero = True
if isNegative2Zero:
    submit = submit.set_index("id", drop=True)
    submit[submit < 0] = 0
    submit = submit.reset_index()

# submit_file = 'xgboost_cv_UseValidPredsIs'+str(actual)+'_Negative2ZeroIs'+str(isNegative2Zero)+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'.csv'
submit_file = 'xgboost_cv_UseValidPredsIs'+str(actual)+'_Negative2ZeroIs'+str(isNegative2Zero)+'_foldday'+str(NUM_FOLD_DAYS)+'_useday'+str(NUM_USE_DAY)+'_lagOlder28Modified.csv'
submit.to_csv(os.path.join('submission_point', submit_file), index=False)